Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (18724, 28, 28) (18724,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (18724, 784) (18724, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [18]:
# logistic model with regularization.

batch_size = 128

graph = tf.Graph()
with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    alpha = tf.constant(.01)

    # Variables.
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + \
        alpha * tf.nn.l2_loss(weights)
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)


Con alpha == 0 es 85.6%.

con alpha == 0.01 es 87.2%

In [19]:
# EXEC.
num_steps = 3001

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))


Initialized
Minibatch loss at step 0: 51.466084
Minibatch accuracy: 10.2%
Validation accuracy: 8.0%
Minibatch loss at step 500: 0.942711
Minibatch accuracy: 82.0%
Validation accuracy: 80.1%
Minibatch loss at step 1000: 0.858202
Minibatch accuracy: 80.5%
Validation accuracy: 81.5%
Minibatch loss at step 1500: 0.784957
Minibatch accuracy: 80.5%
Validation accuracy: 81.4%
Minibatch loss at step 2000: 0.728291
Minibatch accuracy: 84.4%
Validation accuracy: 81.3%
Minibatch loss at step 2500: 0.847330
Minibatch accuracy: 81.2%
Validation accuracy: 81.1%
Minibatch loss at step 3000: 0.857858
Minibatch accuracy: 81.2%
Validation accuracy: 80.2%
Test accuracy: 87.2%


In [4]:
# Neural Network.

from tensorflow.python.framework import ops
ops.reset_default_graph()

batch_size = 128
num_features = image_size * image_size
hidden_layer = 1024

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

graph = tf.Graph()
with graph.as_default():

    # Input data.
    x = tf.placeholder(tf.float32, shape=(None, num_features))
    y_ = tf.placeholder(tf.float32, shape=(None, num_labels))
  
    # Model
    W_nn = weight_variable([num_features, hidden_layer])
    b_nn = bias_variable([hidden_layer])
    h_1 = tf.nn.relu(tf.matmul(x, W_nn) + b_nn)
    W_o = weight_variable([hidden_layer, num_labels])
    b_o = bias_variable([num_labels])
    logits = tf.matmul(h_1, W_o) + b_o
    y_nn = tf.nn.softmax(logits)

    # Train
    alpha = tf.constant(.01)
    loss = -tf.reduce_sum(y_ * tf.log(y_nn)) + \
            alpha * tf.nn.l2_loss(W_nn) + alpha * tf.nn.l2_loss(W_o) 
    optimizer = tf.train.AdamOptimizer(1e-4).minimize(loss)

    # Evaluation
    correct_prediction = tf.equal(tf.argmax(y_nn,1), tf.argmax(y_,1))
    accuracy = (tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) * 100)



In [5]:
num_steps = 20000

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # falls in 0.. size - batchsize.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # feed_dict. key is placeholder, value is the numpy array data.
        feed_dict = {x : batch_data, y_ : batch_labels}
        _, l, __ = session.run([optimizer, loss, y_nn], feed_dict=feed_dict)
        if (step % 500 == 0):
            acc_batch = accuracy.eval(feed_dict={x : batch_data, y_ : batch_labels})
            acc_valid = accuracy.eval(feed_dict={x : valid_dataset, y_ : valid_labels})
            print("Step %5d - Loss %8.4f - Accuracies, batch %.1f%% - validation %.1f%%" %
                  (step, l, acc_batch, acc_valid))
    acc_test = accuracy.eval(feed_dict={x : test_dataset, y_ : test_labels})
    print("Test accuracy: %.1f%%" % acc_test)



Initialized
Step     0 - Loss 545.0051 - Accuracies, batch 9.4% - validation 8.1%
Step   500 - Loss  82.5165 - Accuracies, batch 85.2% - validation 82.5%
Step  1000 - Loss  78.2168 - Accuracies, batch 84.4% - validation 84.1%
Step  1500 - Loss  59.7492 - Accuracies, batch 85.9% - validation 85.3%
Step  2000 - Loss  54.5385 - Accuracies, batch 88.3% - validation 85.9%
Step  2500 - Loss  76.8844 - Accuracies, batch 82.8% - validation 86.3%
Step  3000 - Loss  68.5048 - Accuracies, batch 85.2% - validation 86.6%
Step  3500 - Loss  61.5543 - Accuracies, batch 84.4% - validation 87.1%
Step  4000 - Loss  27.4372 - Accuracies, batch 94.5% - validation 87.3%
Step  4500 - Loss  44.0423 - Accuracies, batch 89.8% - validation 87.6%
Step  5000 - Loss  43.3532 - Accuracies, batch 89.1% - validation 87.7%
Step  5500 - Loss  39.4010 - Accuracies, batch 91.4% - validation 88.1%
Step  6000 - Loss  56.5214 - Accuracies, batch 86.7% - validation 88.2%
Step  6500 - Loss  40.2798 - Accuracies, batch 91.4% -

KeyboardInterrupt: 

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
